In [2]:
import generator
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Activation,LSTM,Bidirectional,Embedding,Conv2D,BatchNormalization,Activation,Masking,LeakyReLU
from tensorflow.keras.activations import tanh,relu
from tensorflow.keras.models import Model
from tensorflow.keras import Model as models
import numpy as np
import librosa
import matplotlib.pyplot as plt
import logging, os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder as one_hot


logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from IPython.display import display
from time import sleep

print('Test 1')
dh = display('Test2',display_id=True)
sleep(1)
dh.update('Test3')

files , json,json_word = generator.gen()
maxi = 1021

def expdim(x,axis):
    return tf.expand_dims(x,axis=axis)

Test 1


'Test3'

In [350]:

#공식 구해야함
'''
maxi = 0
for k,i in enumerate(files):
    wav,_ = librosa.load(files[i]['file_path'],sr=16000)
    mfcc = librosa.feature.mfcc(wav,n_mfcc=40)
    if mfcc.shape[1] > maxi:
        maxi = mfcc.shape[1]
    if k%200==0:
        print(k)
'''
one_hot_json = []
for i,k in enumerate(json):
    one_hot_json.append([json[k][0]])
one_hot_enc=one_hot()
one_hot_enc.fit(one_hot_json)

opt = tf.keras.optimizers.Adam(10e-3)
maxi = 1021
sos_emb = emb_layer(1)

emb_results = [None]*len(files)
mfccs_temp = []
#전체 파일 생성
files_word_num = []

max_len_txt = 0
for k,i in enumerate(files):
    if k % 20 == 0:
        print(k)
    file_word_num = [ json[j][0] for j in files[i]['file_txt']]
    files_word_num.append(file_word_num)
    wav,_ = librosa.load(files[i]['file_path'],sr=16000)
    
    if max_len_txt < len(file_word_num):
        max_len_txt = len(file_word_num)
        
    emb_results[k] = emb_result
    mfcc = np.expand_dims(
                np.expand_dims(
                    librosa.feature.mfcc(wav,n_mfcc=40),axis=-1),axis=0)

    tak = np.zeros((1,40,maxi-mfcc.shape[-2],1),dtype=np.float32)
    mfcc = np.concatenate([mfcc,tak],axis=2)
    mfccs_temp.append(mfcc)
mfccs = np.concatenate(mfccs_temp,axis=0)

embs = []
for j in files_word_num:
    pad = np.zeros([max_len_txt-len(j)])
    embs.append(np.expand_dims(np.concatenate([j,pad],axis=0),axis=0))
embs = np.concatenate(embs,axis=0)
embs.astype(int)

train_mfccs = mfccs[:-270]    
test_mfccs = mfccs[-270:]

train_embs = embs[:-270]    
test_embs = embs[-270:]

train_len = files_word_num[:-270]
test_len = files_word_num[-270:]

json_emb = tf.cast([ json[i][0] for i in json],tf.float32)

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700


In [335]:
embs.shape

TensorShape([2703, 94])

In [382]:
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

def emb_to_word(target):
    cosine_result = cosine_similarity(emb_layer(json_emb),target)
    result = []
    for i in range(cosine_result.shape[1]):
        table = cosine_result[:,i].tolist()
        result.append(json_word[table.index(max(table))])
    return result

def hard_tanh(x):
    return tf.minimum(20.0, tf.maximum(0.0, x)) 
    
get_custom_objects().update({'hard_tanh':Activation(hard_tanh)})
    
class encoder(Model):
    def conv_init(self):
        conv = Sequential([
            Conv2D(32,(21,81),strides=(2,2),padding='valid',input_shape=(40,1021,1)),
            BatchNormalization(-1),
            Activation(hard_tanh),
            Conv2D(32,(10,41),strides=(1,2),padding='valid'),
            BatchNormalization(-1),
            Activation(hard_tanh),
        ])
        
        return conv
    
    def __init__(self):
        super(encoder,self).__init__()
        #self.conv = self.conv_init()
        self.rnn = Sequential([
            Masking(mask_value=0.0),
            LSTM(256,input_shape=(None,40)),
        ])
        
    def call(self,audio):
        audio = tf.transpose(tf.squeeze(audio,3),[0,2,1])
        #x = self.conv(audio)
        #x = self.rnn(tf.squeeze(x,1))
        x = self.rnn(audio)
        return x
        
class seq2(Model):
    def __init__(self, enc, dec):
        super(seq2,self).__init__()
        self.enc = enc
        self.dec = dec
        
    def call(self,audio, txt_emb, txt_len,force_teaching=False):
        enc_out = self.enc(audio)
        dec_out = self.dec(enc_out,txt_emb, txt_len,force_teaching)
        
        return dec_out
    
    def predict(self, audio, sos_emb):
        
        enc_out = self.enc(audio)
        enc_out = tf.expand_dims(enc_out,axis=0)
        
        sos_emb = tf.expand_dims(tf.expand_dims(sos_emb,axis=0),axis=0)
        
        predict_emb = tf.concat([sos_emb,enc_out],axis=2)
        
        #init
        #predict_emb = self.dec.mask(predict_emb)
        output, h , c = self.dec.rnn(predict_emb)
        cur_word = tf.expand_dims(tf.expand_dims(output[0,-1,:],axis=0),axis=0)
        output_dim = output
        
        hidden = [h,c]
        stop_sign = False
        result = [sos_emb]
        
        k=0
        while not stop_sign:
            
            '''if k == 3:
                break'''
            enc_outs = tf.tile(enc_out,[1,output_dim.shape[1],1])
            predict_emb = tf.concat([output_dim,enc_outs],axis=2)
            
            #predict_emb = tf.concat([cur_word,enc_out],axis=2)
            
            #predict_emb = self.dec.mask(tf.concat([cur_word,enc_out],axis=2))
            
            print(predict_emb.shape)
            output , h ,c = self.dec.rnn(
                predict_emb,
                initial_state = hidden
            )
            hidden = [h,c]
            
            pre_word = cur_word
            cur_word = tf.expand_dims(tf.expand_dims(output[0,-1,:],axis=0),axis=0)
            
            #print(pre_word,cur_word)
            #print(cosine_similarity(tf.squeeze(pre_word,0),tf.squeeze(cur_word,0)))
            
            output_dim = tf.concat([output_dim,cur_word],axis=1)
            print(output_dim.shape,output.shape)
            
            result.append(cur_word)
            
            if len(result) > 93:
                stop_sign = True
            k+=1
        
        return result
        
        

class decoder(Model):
    def __init__(self):
        super(decoder,self).__init__()
        
        self.emb = Embedding(len(json),512)
        self.mask = Masking(mask_value=0.0)
        self.rnn = LSTM(512,input_shape=(None,768),return_sequences=True,return_state=True)
        
    def call(self,enc_out,embs,txt_len,force_teaching=False):
        
        embs = self.emb(embs)
        re_all=[]
        padded = []
        for i,k in enumerate(txt_len):
            tiled = tf.tile(tf.expand_dims(tf.expand_dims(enc_out[i],axis=0),axis=0),[1,len(k),1])
            pad = tf.convert_to_tensor(np.zeros((1,94-len(k),256)),tf.float32)
            tiled = tf.concat([tiled,pad],axis=1)
            padded.append(tiled)
        padded = tf.concat(padded,axis=0)
        
        if force_teaching:
            
            cats = tf.concat([embs,padded],axis=2)
            cat = self.mask(cats[:,0,:])
            result,h,c = self.rnn(tf.expand_dims(cat,axis=1))
            hidden = [h,c]
            re_all.append(result)
            
        else:
            inputs = tf.tile(tf.expand_dims(tf.expand_dims(sos_emb,axis=0),axis=0),[enc_out.shape[0],94,1])
            cats = tf.concat([inputs,padded],axis=2)
            
            cat = self.mask(cats[:,0,:])
            result,h,c = self.rnn(tf.expand_dims(cat,axis=1))
            hidden = [h,c]
            re_all.append(result)
            
        for i in range(1,cats.shape[1]):
            step_cat = tf.expand_dims(cats[:,i,:],axis=1)
            step_cat = self.mask(step_cat)
            result,h,c = self.rnn(result,initial_state=hidden)
            hidden = [h,c]
            re_all.append(result)
            
        re_all = tf.concat(re_all,axis=1)
        
        return re_all


In [380]:
def cosine_similarity_3d(matrix, vector):
    ''' Computes cosine similarity of a given vector with vector rows from matrix'''

    # normalize input
    mat = tf.nn.l2_normalize(matrix,2)
    mat2 = tf.nn.l2_normalize(tf.transpose(vector,[0,2,1]),1)

    # multiply row i with row j using transpose
    similar = tf.matmul(mat[:,:],mat2[:,:])

    return similar

def transfer_one_hot(sens):
    one_hot_emb = []
    for sen in sens:
        tr_oh = np.expand_dims(one_hot_enc.transform(train_embs[0].reshape(-1,1)).toarray(),0)
        one_hot_emb.append(tr_oh)
    one_hot_sens = np.concatenate(one_hot_emb,axis=0)
    
    return one_hot_sens

In [383]:
enc = encoder()
dec = decoder()
model = seq2(enc,dec)
opt = tf.keras.optimizers.Adam(10e-5)

In [385]:
#train

train_loss = tf.keras.metrics.Mean(name='train_loss')
df = display('start',display_id=True)

with tf.device(f'GPU:{0}'):
    for iter_num in range(1000):
        for i in range(train_embs.shape[0]//30):
            #batch_emb = transfer_one_hot(train_embs[i*30:(i+1)*30])
            batch_emb = train_embs[i*30:(i+1)*30]
            batch_mfcc = train_mfccs[i*30:(i+1)*30]
            batch_len = train_len[i*30:(i+1)*30]
            
            with tf.GradientTape() as tape:
                
                result = model(batch_mfcc,batch_emb,batch_len,True)
                loss=tf.reduce_mean(tf.reduce_sum(tf.math.sqrt(tf.reduce_sum(tf.math.pow(batch_emb-result,2),2)),1))
                '''loss = tf.reduce_mean(tf.math.sqrt(
                    tf.reduce_sum(tf.math.pow(cosine_similarity_3d(batch_emb,result),2),axis=1)))'''
                train_loss(loss)
                
                #print(emb_to_word(tf.squeeze(result[0])))
            grad = tape.gradient(loss,model.trainable_weights)
            opt.apply_gradients(zip(grad,model.trainable_weights))
            
        print(train_loss.result().numpy())
        df.update(tf.math.sqrt(tf.reduce_sum(tf.math.pow(batch_emb-result,2),2))[20])
        print(emb_to_word(tf.squeeze(result[20])),emb_to_word(batch_emb[20]))
                 

'start'

(30, 94)
(30, 94, 512)
tf.Tensor(
[[-7.6158482e-01  1.0411030e-14  2.7118299e-12 ... -1.9958385e-18
  -3.1354685e-21  6.9254734e-06]
 [-7.6155180e-01  1.3594986e-13  3.4968838e-13 ... -3.3597533e-16
  -1.2257518e-18 -8.4724779e-06]
 [-7.6102513e-01  1.1142798e-11  2.7721573e-12 ... -6.0346200e-13
  -3.2247765e-17 -5.5418451e-07]
 ...
 [-7.6159406e-01  8.3390884e-13  1.0125324e-12 ... -1.4875412e-18
  -1.4264918e-21 -7.6922124e-06]
 [-7.6145482e-01  4.6835615e-13  2.5929089e-11 ... -3.8370199e-15
  -4.2875439e-19  1.3828137e-06]
 [-7.6157737e-01  1.0215803e-13  7.4864706e-12 ... -5.2996433e-15
  -9.3295054e-19 -2.0693510e-06]], shape=(30, 256), dtype=float32)
1
['waiting', 'shake', 'move', 'dispatch', 'uncle', 'confined', 'sledge', 'posturing', 'shrieks', 'uncle', 'posturing', 'waiting', 'plays', 'distanced', 'posturing', 'shake', 'confined', 'move', 'uncle', 'posturing', 'interests', 'idea', 'enjoyed', 'alaska', 'confined', 'use', 'waiting', 'dowson', 'ebbed', 'rises']
[['logical'], ['

KeyboardInterrupt: 

In [ ]:
result = model.predict(expdim(test_mfccs[1],0),sos_emb)

In [188]:
print(emb_to_word(tf.squeeze(result)))

['<sos>', 'neither', 'stab', 'stab', 'stab', 'stab', 'stab', 'stab', 'stab', 'stab', 'stab', 'stab', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later', 'later']
